## COMP5328 - Advanced Machine Learning
## Assignment 2: Title
----------------------------------------------------------------------------------------

In [468]:
# Common imports
import os
import glob
import numpy as np
import json
import time
import pandas as pd
from datetime import datetime
import tabulate

# Ploting
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torch.optim as optim
import random
import scipy.stats as st

In [469]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ Using Apple MPS:", device)
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("✅ Using NVIDIA CUDA:", device)
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU:", device)

✅ Using Apple MPS: mps


In [470]:
# Experiment variables
# Common
num_classes=3
dataset_folder = 'data/'

cifar_dataset = dataset_folder+'CIFAR.npz'
MNISTO3_dataset = dataset_folder+'FashionMNIST0.3.npz'
MNISTO6_dataset = dataset_folder+'FashionMNIST0.6.npz'


DATA_PATHS = {
    'fashion03': MNISTO3_dataset,
    'fashion06': MNISTO6_dataset,
    'cifar':     cifar_dataset
}

losses = ['forward','gce', 'forwardGCE', 'trevision']
losses = ['forwardGCE', 'trevision']
datasets = ['cifar', "fashion03", "fashion06"]
base = {
    "runs":10,
    "epochs": 10,
    "loss":'forward',
    "batch_size":256,
    "q":0.6,
    "est_epochs":25,
    "beta":5e-4,
    "lr": 1e-2,
    "device":str(device)
}

known_T_fashion_03 = np.array(  [[0.7,0.3,0.0],
                                [0.0,0.7,0.3],
                                [0.3,0.0,0.7]], dtype=np.float32)

known_T_fashion_06 = np.array(  [[0.4,0.3,0.3],
                                [0.3,0.4,0.3],
                                [0.3,0.3,0.4]], dtype=np.float32)



In [471]:
# Common Functions
def pick_known_T(tag):
    if tag == 'fashion03':
        return known_T_fashion_03
    elif tag == 'fashion06':
        return known_T_fashion_06
    else:
        return None

def set_seed(seed=0):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def sanity(T, name, dataset):
    print(f"\n{name}")

    # ensure tensor and move to CPU for printing
    if isinstance(T, np.ndarray):
        T = torch.tensor(T, dtype=torch.float32)
    T = T.detach().cpu()

    print("row sums:", T.sum(dim=1))
    print("col sums:", T.sum(dim=0))
    # Check T is row-stochastic and reasonable

    print("cond(T):", float(torch.linalg.cond(T).item()))



    # pick true transition
    if dataset == 'fashion03':
        T_true = torch.tensor(known_T_fashion_03, dtype=torch.float32)
    elif dataset == 'fashion06':
        T_true = torch.tensor(known_T_fashion_06, dtype=torch.float32)
    else:
        T_true = None

    if T_true is not None:
        T_true = T_true.to(T.device)
        fro = torch.norm(T - T_true, p='fro').item()
        mae = torch.mean(torch.abs(T - T_true)).item()
        print(f"Fro: {fro:.6f}")
        print(f"MAE: {mae:.6f}")


def load_npz(path):
    d = np.load(path)
    Xtr, Str = d['Xtr'], d['Str']
    Xts, Yts = d['Xts'], d['Yts']
    return Xtr, Str, Xts, Yts

## 1. Load Dataset

### 1.0 Data Folder

In [472]:
# The structure of data folder.
!ls -l data

total 172688
-rw-r--r--@ 1 jamie.saunders  staff  55440974 Oct  4  2019 CIFAR.npz
-rw-r--r--@ 1 jamie.saunders  staff  16485974 Oct 10  2021 FashionMNIST0.3.npz
-rw-r--r--@ 1 jamie.saunders  staff  16485974 Oct 10  2021 FashionMNIST0.6.npz


In [473]:
# Code taken from Week10 Tutorial

# A helper class, it is used as an input of the DataLoader object.
class DatasetArray(Dataset):
    r"""This is a child class of the pytorch Dataset object."""
    def __init__(self, data, labels=None, transform=None):
        if labels != None:
            self.data_arr = np.asarray(data).astype(np.float32)
            self.label_arr = np.asarray(labels).astype(np.long)
        else:
            tmp_arr = np.asarray(data)
            self.data_arr = tmp_arr[:,:-1].astype(np.float32)
            self.label_arr = tmp_arr[:,-1].astype(np.long)
        self.transform = transform
        
    def __len__(self):
        return len(self.data_arr)
    
    def __getitem__(self, index):
     
        data = self.data_arr[index]
        label = self.label_arr[index]
        
        if self.transform is not None:
            data = self.transform(data)
            
        return (data, label)
    
    
# Splitting the data into three parts.
def train_val_test_random_split(data, fracs=[0.7,0.1,0.2]):
    r"""Split the data into training, validation and test set.
    Args:
        fracs: a list of length three
    """
    assert len(fracs) == 3
    assert sum(fracs) == 1
    assert all(frac > 0 for frac in fracs)
    n = len(data)
    subset_lens = [int(n*frac) for frac in fracs]
    idxs = list(range(n))
    random.shuffle(idxs)
    data = np.array(data)
    new_data = []
    start_idx = 0
    for subset_len in subset_lens:
        end_idx = start_idx + subset_len
        cur_idxs = idxs[start_idx:end_idx]
        new_data.append(data[cur_idxs,:].tolist())
        start_idx = end_idx
    return new_data

# Preparation of the data for training, validation and testing a pytorch network. 
# Note that the test data is not in use for this lab.
def get_loader(batch_size =128, num_workers = 0, train_val_test_split = [0.7,0.1,0.2], data=None):
    r"""This function is used to read the data file and split the data into three subsets, i.e, 
    train data, validation data and test data. Their corresponding DataLoader objects are returned."""
    
    [train_data, val_data, test_data] = train_val_test_random_split(data, fracs = train_val_test_split)

    train_data = DatasetArray(data = train_data)
    val_data = DatasetArray(data = val_data)
    test_data = DatasetArray(data = test_data)

    #The pytorch built-in class DataLoader can help us to shuffle the data, draw mini-batch,
    #do transformations, etc. 
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=100,
        shuffle=False,
        num_workers=num_workers,
    )

    test_loader = DataLoader(
        test_data,
        batch_size=100,
        num_workers=num_workers,
        shuffle=False,
    )
    return train_loader, val_loader, test_loader

In [474]:
class NpzDataset(Dataset):
    def __init__(self, X, y, is_cifar=False):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)
        self.is_cifar = is_cifar

        # Normalize to [0,1]
        self.X = self.X / 255.0 if self.X.max() > 1.0 else self.X

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = self.X[idx]
        if x.ndim == 1:
            # flat; try to infer shape 28x28 or 32x32x3
            if x.size == 28*28:
                x = x.reshape(1, 28, 28)
            elif x.size == 32*32*3:
                x = x.reshape(3, 32, 32)
            else:
                raise ValueError("Unknown flat image shape: {}".format(x.shape))
        else:
            # (H,W) or (H,W,C)
            if x.ndim == 2:
                x = x[None, ...]  # to (1,H,W)
            elif x.ndim == 3:
                # assume HWC -> CHW
                x = np.transpose(x, (2, 0, 1))
            else:
                raise ValueError(f"Unexpected image dims: {x.shape}")
        return torch.from_numpy(x), torch.tensor(self.y[idx])



def make_loaders(Xtr, Str, batch_size=128, seed=0, test_size=0.2):
    # 80/20 split each repetition
    X_tr, X_val, y_tr, y_val = train_test_split(
        Xtr, Str, test_size=test_size, random_state=seed, stratify=Str
    )

    is_cifar = (X_tr.shape[-1] == 3) if X_tr.ndim == 4 else (X_tr.shape[-1] == 32*32*3)

    train_ds = NpzDataset(X_tr, y_tr, is_cifar)
    val_ds   = NpzDataset(X_val, y_val, is_cifar)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader, is_cifar

def make_test_loader(Xts, Yts, batch_size=256):
    is_cifar = (Xts.shape[-1] == 3) if Xts.ndim == 4 else (Xts.shape[-1] == 32*32*3)
    test_ds = NpzDataset(Xts, Yts, is_cifar)
    return DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=0)

In [475]:
class ForwardCorrectedCE(nn.Module):
    """
    Forward loss correction: minimizes CE between T^T p and noisy labels.
    T: class-transition matrix where T[i,j] = P(S=j | Y=i). Shape [C,C].
    """
    def __init__(self, T):
        super().__init__()
        self.register_buffer('T', T)  # [C,C]

    def forward(self, logits, y_noisy):
        # logits -> p(y|x)
        p = F.softmax(logits, dim=1)  # [B,C]
        # mix via T^T
        mixed = torch.clamp(p @ self.T, 1e-6, 1.0)
        log_mixed = torch.log(mixed)
        return F.nll_loss(log_mixed, y_noisy)

class ForwardCorrectedGCE(nn.Module):
    def __init__(self, T, q):
        super().__init__()
        self.register_buffer('T', T)
        self.q = q
    def forward(self, logits, y_noisy):
        p_noisy = torch.clamp(F.softmax(logits, dim=1) @ self.T, 1e-6, 1.0)
        p_s = p_noisy.gather(1, y_noisy.view(-1,1)).clamp(1e-6,1.0)
        return (-(p_s.log()) if self.q==1.0 else (1 - p_s**self.q)/self.q).mean()

class GeneralizedCrossEntropy(nn.Module):
    """
    GCE loss: L_q(p, y) = (1 - p_y^q) / q, with q in (0,1].
    q→1 recovers CE; smaller q is more robust to label noise.
    """
    def __init__(self, q):
        super().__init__()
        assert 0 < q <= 1
        self.q = q

    def forward(self, logits, y):
        p = F.softmax(logits, dim=1)
        p_y = p.gather(1, y.view(-1,1)).clamp(min=1e-6, max=1.0)
        if self.q == 1.0:
            return -torch.log(p_y).mean()
        return ((1 - p_y.pow(self.q)) / self.q).mean()
    
    
class TRevisionLoss(nn.Module):
    """
    Joint T-revision loss: learns transition matrix T along with model weights.
    Combines forward correction with a regularization toward identity (or prior).
    """
    def __init__(self, device, C=3, q=0.7, lambda_reg=1e-3, T0=None):
        super().__init__()
        self.q = q
        self.lambda_reg = lambda_reg
        eps = 1e-6
        # ------------------------------
        # Initialize T0
        # ------------------------------
        if T0 is None:
            # Start from identity (clean labels)
            T0 = torch.eye(C, device=device, dtype=torch.float32)
        else:
            # Safely handle both numpy arrays and tensors
            T0 = torch.as_tensor(T0, dtype=torch.float32, device=device)
            # Ensure it's detached and cloned to avoid warnings
            T0 = T0.detach().clone()

        # Clamp to ensure positivity and numerical stability
        T0 = torch.clamp(T0, eps, 1.0)

        # ------------------------------
        # Logits parameterization
        # ------------------------------
        # Learn unnormalized log-values; softmax over columns later gives valid T
        self.logits = nn.Parameter(torch.log(T0))



    def forward(self, logits, y):
        """
        logits: model outputs (B, C)
        y: noisy labels (B,)
        """
        C = logits.size(1)
        T = torch.softmax(self.logits, dim=1)  # ensure column-stochastic
        p = F.softmax(logits, dim=1)
        p_noisy = torch.clamp(p @ T, 1e-6, 1.0)
        p_s = p_noisy.gather(1, y.view(-1, 1)).clamp(1e-6, 1.0)

        if self.q == 1.0:
            loss = -torch.log(p_s)
        else:
            loss = (1 - p_s.pow(self.q)) / self.q
        loss = loss.mean()

        # regularize T toward identity or prior
        T_prior = torch.eye(C, device=T.device)
        reg = self.lambda_reg * torch.norm(T - T_prior, p='fro')
        return loss + reg


In [476]:
def conv_block(cin, cout):
    return nn.Sequential(
        nn.Conv2d(cin, cout, 3, padding=1),
        nn.BatchNorm2d(cout),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2)
    )

class SmallCNN28(nn.Module):
    """For 1×28×28 images."""
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            conv_block(1, 32),  # 14x14
            conv_block(32, 64), # 7x7
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 3)
        )
    def forward(self, x):
        return self.net(x)


class SmallCNNCifar(nn.Module):
    """Improved for CIFAR (3×32×32)."""
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            conv_block(3, 64),     # 16x16
            conv_block(64, 128),   # 8x8
            conv_block(128, 256),  # 4x4
            nn.Dropout(0.3)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*4*4, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 3)
        )
    def forward(self, x):
        return self.classifier(self.features(x))


def make_model(is_cifar, device=None):
    model = SmallCNNCifar() if is_cifar else SmallCNN28()
    if device is not None:
        model = model.to(device)
    return model

In [477]:
@torch.no_grad()
def accuracy(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        pred = logits.argmax(dim=1)
        correct += (pred == yb).sum().item()
        total += yb.numel()
    return correct / max(total, 1)


def train_one_epoch(model, loader, optimizer, criterion, device=device):
    model.train()
    total_loss = 0.0
    n_samples = 0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        batch_size = yb.size(0)
        total_loss += loss.detach().item() * batch_size
        n_samples += batch_size

    mean_loss = total_loss / n_samples
    return mean_loss


def fit_model(model, train_loader, val_loader, device, loss_name, T, q, beta=0.2, epochs=10, lr=1e-3):

    model.to(device)
    if T is not None:
        if not isinstance(T, torch.Tensor):
            T = torch.tensor(T, dtype=torch.float32, device=device)
        else:
            T = T.detach().clone().to(torch.float32).to(device)
        
    if loss_name == 'forward':
        assert T is not None, "Forward correction requires known/estimated T"
        criterion = ForwardCorrectedCE(T)
    elif loss_name == 'forwardGCE':
        assert T is not None, "Forward correction requires known/estimated T"
        criterion = ForwardCorrectedGCE(T, q=q)
    elif loss_name == 'trevision':
        assert T is not None, "T-revision requires known/estimated T"
        criterion = TRevisionLoss(C=3,device=device, q=q, lambda_reg=beta, T0=T).to(device)
    else:
        criterion = GeneralizedCrossEntropy(q=q)
        
    criterion = criterion.to(device)

    if isinstance(criterion, TRevisionLoss):
        optimizer = optim.Adam(
            list(model.parameters()) + list(criterion.parameters()),
            lr=lr, weight_decay=1e-4
        )
    else:
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

    best_val = -np.inf
    best_state = None
    loss_arr = []

    for _ in range(epochs):
        mean_loss, mean_acc, loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        # early stopping on val accuracy (cheap)
        val_acc = accuracy(model, val_loader, device)
        if val_acc > best_val:
            best_val = val_acc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        loss_arr.append(loss)
        
    if best_state is not None:
        model.load_state_dict(best_state)

    return model, loss_arr

In [478]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def train_one_epoch(model, loader, optimizer, criterion=None, device=None):
    # ---- device setup ----
    if device is None:
        device = (
            "cuda" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu"
        )
    device = torch.device(device)

    # move model to device
    model.to(device)
    model.train()

    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss(reduction='none')  # per-sample losses

    all_losses = []   # for histogram
    batch_metrics = []  # (batch_idx, acc, mean_loss)

    for batch_idx, (xb, yb) in enumerate(loader, 1):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad(set_to_none=True)

        logits = model(xb)
        per_sample_loss = criterion(logits, yb)

        # ensure scalar loss
        if per_sample_loss.ndim > 0:
            loss = per_sample_loss.mean()
            all_losses.extend(per_sample_loss.detach().cpu().tolist())
        else:
            loss = per_sample_loss
            all_losses.append(loss.detach().cpu().item())

        loss.backward()
        optimizer.step()

        preds = logits.argmax(dim=1)
        acc = (preds == yb).float().mean().item()
        batch_metrics.append((batch_idx, acc, loss.item()))
        #print(f"Batch {batch_idx:03d}: loss={loss.item():.4f}, acc={acc*100:.2f}%")
        

    # ---- epoch summary ----
    mean_loss = sum([l for _,_,l in batch_metrics]) / len(batch_metrics)
    mean_acc = sum([a for _,a,_ in batch_metrics]) / len(batch_metrics)
    print(f"\nEpoch summary: mean_loss={mean_loss:.4f}, mean_acc={mean_acc*100:.2f}%")
    with torch.no_grad():
        total_norm = 0
        for p in model.parameters():
            total_norm += p.grad.data.norm(2).item() if p.grad is not None else 0
    print(f"Grad norm: {total_norm:.4f}")

    return mean_loss, mean_acc, all_losses


In [479]:
@torch.no_grad()
def predict_proba(model, loader, device):

    model.eval()
    p_list, y_list = [], []
    for xb, yb in loader:
        xb = xb.to(device)
        output = model(xb)
        p = F.softmax(output, dim=1)
        p_list.append(p)
        y_list.append(yb.to(device))
    return torch.cat(p_list), torch.cat(y_list)


In [480]:
import matplotlib.pyplot as plt
import numpy as np

def plot_run_performance(all_acc, all_mean_loss, loss_name, dataset):
    runs = np.arange(1, len(all_acc) + 1)

    fig, ax1 = plt.subplots(figsize=(8,5))
    ax2 = ax1.twinx()

    # Accuracy (left axis)
    ax1.plot(runs, np.array(all_acc)*100, 'o-', color='tab:blue', label='Test Accuracy (%)')
    ax1.set_xlabel('Run')
    ax1.set_ylabel('Accuracy (%)', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.set_ylim(0, 100)

    # Loss (right axis)
    ax2.plot(runs, all_mean_loss, 's--', color='tab:red', label='Mean Train Loss')
    ax2.set_ylabel('Loss', color='tab:red')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    plt.title(f"{dataset.upper()} — {loss_name} Performance Across Runs")
    fig.tight_layout()

    # combine legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines + lines2, labels + labels2, loc='best')

    plt.show()


In [481]:
def estimate_transition_anchor(t, train_loader, is_cifar, q, device, epochs):
    """
    Simple anchor/confident-example estimator (Patrini et al., 2017 style):
    1) Train a base classifier on noisy data.
    2) Get p(y|x) on training set.
    3) For each clean class i, find indices whose predicted argmax == noisy label == i and with high confidence.
    4) For those indices, estimate column i of T as average of empirical noisy label distribution given model predicts i.
    Here: since we only have noisy labels S, we approximate T[:, i] ≈ E[ onehot(S) | argmax p = i, p_i >= τ ].
    Normalize columns to sum to 1.
    """

    model = make_model(is_cifar).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    criterion = GeneralizedCrossEntropy(q=q)
    # quick warmup training on noisy labels
    for _ in range(epochs):
        train_one_epoch(model, train_loader, optimizer, criterion, device=device) 
        
    #print('devise - estimation')   
    #print(next(model.parameters()).device)


    # collect probs & noisy labels
    p_arr, y_noisy = predict_proba(model, train_loader, device)

    # ensure both are torch tensors on same device
    if not torch.is_tensor(p_arr):
        p_arr = torch.tensor(p_arr, dtype=torch.float32, device=device)
    if not torch.is_tensor(y_noisy):
        y_noisy = torch.tensor(y_noisy, dtype=torch.long, device=device)
    else:
        y_noisy = y_noisy.to(device, dtype=torch.long)

    preds = p_arr.argmax(dim=1)
    maxp = p_arr.max(dim=1).values

    C = num_classes
    T = t

    if isinstance(T, np.ndarray):
        T = torch.tensor(T, dtype=torch.float32, device=device)
    elif torch.is_tensor(T):
        T = T.to(device)
        
    # choose class-wise thresholds based on quantiles for stability
    for i in range(C):
        idx = (preds == i).nonzero(as_tuple=True)[0]
        if idx.numel() == 0:
            T[i, :] = torch.ones(C, device=device) / C
            continue
        # high-confidence subset (top 30% by p_i)
        conf = maxp[idx]
        if conf.numel() > 50:
            tau = torch.quantile(conf, 0.7).item()
        else:
            tau = conf.min().item()
        keep = idx[conf >= tau]
        if keep.size == 0:
            keep = idx
        # empirical distribution of noisy labels among confident examples
        hist = torch.bincount(y_noisy[keep], minlength=C).float()
        if hist.sum() == 0:
            T[i, :] = torch.ones(C, device=device) / C
        else:
            T[i, :] = hist / hist.sum()

    # normalize rows to sum to 1
    T = torch.clamp(T, min=1e-8)
    T = T / T.sum(dim=1, keepdim=True)

    return T.detach().to(device)



In [482]:
def run_once(args, T, seed):
    set_seed(seed)
    q = args['q']
    device = torch.device(args['device'])

    C=3
    # load data
    Xtr, Str, Xts, Yts = load_npz(DATA_PATHS[args['dataset']])

    # loaders for this split
    train_loader, val_loader, is_cifar = make_loaders(Xtr, Str, batch_size=args['batch_size'], seed=seed)
    test_loader = make_test_loader(Xts, Yts, batch_size=512)

    xb, yb = next(iter(train_loader))
    print("Batch shape:", xb.shape, "min/max:", xb.min().item(), xb.max().item())
  
    # choose model
    model = make_model(is_cifar).to(device)


    # fit
    model, loss_arr = fit_model(
        model,
        train_loader,
        val_loader,
        device,
        loss_name=args['loss'],
        T=T,
        q=q,
        beta=args['beta'],
        epochs=args['epochs'],
        lr=args['lr'],
    )

    all_loss_arrays = loss_arr
    
    # evaluate on clean test set
    test_acc = accuracy(model, test_loader, device)
    return float(test_acc), (T if T is not None else None), loss_arr

In [483]:
# Define config
now = datetime.now()
now = now.strftime("%Y-%m-%d-%H_%M")
folder = "results_"+now
if os.path.exists(folder) and os.path.isdir(folder):
    os.rmdir(folder)
    os.mkdir(folder)
else:
    os.mkdir(folder)
# create each cfg
estimate = ['anchor', 'None']
qs = [0.7]
configs = []
for q in qs:
    for i, ds in enumerate(datasets):
        if ds == 'cifar':
            is_cifar = True
        else:
            is_cifar = False
        
        for loss in losses:
            for t in estimate:
                cfg = {**base, "dataset": ds, "loss":loss, "estimate_T":t, "q":q, "is_cifar":is_cifar}
                if t=='None' and ds=='cifar':
                    print(f"Skipping dataset: {ds}, estimate_T:{t}, loss:{loss}, q:{q}")
                else:
                    print(f"dataset: {ds}, estimate_T:{t}, loss:{loss}, q:{q}")
                    configs.append(cfg)



dataset: cifar, estimate_T:anchor, loss:forwardGCE, q:0.7
Skipping dataset: cifar, estimate_T:None, loss:forwardGCE, q:0.7
dataset: cifar, estimate_T:anchor, loss:trevision, q:0.7
Skipping dataset: cifar, estimate_T:None, loss:trevision, q:0.7
dataset: fashion03, estimate_T:anchor, loss:forwardGCE, q:0.7
dataset: fashion03, estimate_T:None, loss:forwardGCE, q:0.7
dataset: fashion03, estimate_T:anchor, loss:trevision, q:0.7
dataset: fashion03, estimate_T:None, loss:trevision, q:0.7
dataset: fashion06, estimate_T:anchor, loss:forwardGCE, q:0.7
dataset: fashion06, estimate_T:None, loss:forwardGCE, q:0.7
dataset: fashion06, estimate_T:anchor, loss:trevision, q:0.7
dataset: fashion06, estimate_T:None, loss:trevision, q:0.7


In [484]:
def stabilize_T(T, alpha=0.7):
    """
    Blend estimated T with identity to avoid near-singular/over-uniform matrices.
    Keeps rows summing to 1.
    """
    C = T.size(0)
    I = torch.eye(C, device=T.device, dtype=T.dtype)
    T = alpha * T + (1 - alpha) * I
    T = torch.clamp(T, 1e-3, 0.999)
    T = T / T.sum(dim=1, keepdim=True)
    return T

In [ ]:
results_rows = []
for cfg in configs:
      # collect summary rows for TSV
    dataset_file = f"{folder}/{cfg['dataset']}_results.json"
    if os.path.exists(dataset_file):
        with open(dataset_file, 'r') as f:
            try:
                all_data = json.load(f)
            except json.JSONDecodeError:
                all_data = []
    else:
        all_data = []


    all_acc = []
    all_mean_loss = []
    all_loss_arrays = []
    last_T = None
    t_arr = []

        # load data
    Xtr, Str, Xts, Yts = load_npz(DATA_PATHS[cfg['dataset']])
        # Transition matrix
    T = None
    train_loader, val_loader, is_cifar = make_loaders(Xtr, Str, batch_size=cfg['batch_size'], seed=0)
   
    if cfg['estimate_T']=='anchor':
        T = torch.zeros((C, C), dtype=torch.float32, device=device)
        T = estimate_transition_anchor(T, train_loader, is_cifar, q, device=device, epochs=cfg['est_epochs'])
        sanity(T, 'Est T After Anchor point', cfg['dataset'])
        
        print("Estimated T:")
        print(T.cpu().numpy())
    else:
        T = pick_known_T(cfg['dataset'])
        if T is None:
            raise ValueError("Forward loss selected but no known T for this dataset; use --estimate_T.")
            

    if isinstance(T, np.ndarray):
        T = torch.tensor(T, dtype=torch.float32, device=device)

    if T is not None:
        if not torch.is_tensor(T):
            T = torch.tensor(T, dtype=torch.float32, device=device)
        else:
            T = T.to(device=device, dtype=torch.float32)

        # Ensure row-stochastic
        T = torch.clamp(T, 1e-6, 1.0)
        T = T / T.sum(dim=1, keepdim=True)

        # Stabilize
        T = stabilize_T(T, alpha=0.7)
        
    # get true T if known
    T_true = pick_known_T(cfg['dataset'])
    has_T_true = T_true is not None

    for r in range(cfg['runs']):
        acc, T, loss_arr = run_once(cfg, T, seed=1000+r)

        all_acc.append(acc)
        all_loss_arrays.append(loss_arr)
        all_mean_loss.append(np.mean(loss_arr))
        
        if cfg['estimate_T'] or cfg['dataset'] == 'cifar':
            t_arr.append(T)
        last_T = T if T is not None else last_T

    plot_run_performance(all_acc, all_mean_loss, cfg['loss'], cfg['dataset'])
    mean_acc = float(np.mean(all_acc))
    std_acc  = float(np.std(all_acc))
    mean_loss = float(np.mean(all_mean_loss))
    std_loss  = float(np.std(all_mean_loss))

    # compute metrics vs true T (if available)
    fro_err = mae_err = rre_err = None
    if has_T_true and last_T is not None:
        T_true = np.array(T_true)
        T_est = np.array(last_T)
        fro_err = float(np.linalg.norm(T_est - T_true, 'fro'))
        mae_err = float(np.mean(np.abs(T_est - T_true)))
        rre_err = float(fro_err / np.linalg.norm(T_true, 'fro'))

    # build summary object
    summary = {
        'cfg': cfg,
        'dataset': cfg['dataset'],
        'loss': cfg['loss'],
        'estimate_T': cfg['estimate_T'],
        'epochs': cfg['epochs'],
        'runs': cfg['runs'],
        'mean_test_acc': mean_acc,
        'std_test_acc': std_acc,
        'mean_train_loss': mean_loss,
        'std_train_loss': std_loss,
        'per_run_acc': all_acc,
        'per_run_mean_loss': all_mean_loss,
        'per_run_loss_arrays': all_loss_arrays,
        'last_estimated_T': last_T,
        't_arr': t_arr,
        'fro_error': fro_err,
        'mae_error': mae_err,
        'rre_error': rre_err
    }

    print("=" * 72)
    print(f"{cfg['dataset']} | {cfg['loss']} | mean±std acc: {mean_acc*100:.2f}±{std_acc*100:.2f}% | mean loss: {mean_loss:.4f}")

    if fro_err is not None:
        print(f"T-error: Fro {fro_err:.4f}, MAE {mae_err:.4f}, RRE {rre_err:.4f}")

    all_data.append(summary)

    with open(dataset_file, 'w') as f:
        json.dump(all_data, f, indent=2)
    print(f"Appended results for {cfg['loss']} → {dataset_file}")

    # ➕ Add a summary row for TSV
    results_rows.append({
        'dataset': cfg['dataset'],
        'loss': cfg['loss'],
        'estimate_T': cfg['estimate_T'],
        'epochs': cfg['epochs'],
        'runs': cfg['runs'],
        'mean_test_acc': round(mean_acc * 100, 2),
        'std_test_acc': round(std_acc * 100, 2),
        'mean_train_loss': round(mean_loss, 4),
        'std_train_loss': round(std_loss, 4),
        'fro_error': None if fro_err is None else round(fro_err, 4),
        'mae_error': None if mae_err is None else round(mae_err, 4),
        'rre_error': None if rre_err is None else round(rre_err, 4)
    })

# After each config loop, write TSV summary
if results_rows:
    df = pd.DataFrame(results_rows)
    tsv_path = os.path.join(folder, "summary_results.tsv")
    df.to_csv(tsv_path, sep='\t', index=False)
    print(f"\n✅ Wrote table summary to {tsv_path}")
    print(df.to_markdown(index=False))



Epoch summary: mean_loss=0.8192, mean_acc=34.53%
Grad norm: 0.8414

Epoch summary: mean_loss=0.7613, mean_acc=37.11%
Grad norm: 0.7713

Epoch summary: mean_loss=0.7578, mean_acc=37.38%
Grad norm: 0.7263

Epoch summary: mean_loss=0.7551, mean_acc=37.51%
Grad norm: 0.8353

Epoch summary: mean_loss=0.7518, mean_acc=38.84%
Grad norm: 0.9744

Epoch summary: mean_loss=0.7508, mean_acc=38.79%
Grad norm: 0.7969

Epoch summary: mean_loss=0.7509, mean_acc=38.43%
Grad norm: 1.0075

Epoch summary: mean_loss=0.7486, mean_acc=38.48%
Grad norm: 0.9003

Epoch summary: mean_loss=0.7460, mean_acc=39.38%
Grad norm: 0.8948

Epoch summary: mean_loss=0.7461, mean_acc=39.11%
Grad norm: 0.7852

Epoch summary: mean_loss=0.7435, mean_acc=39.99%
Grad norm: 0.9658

Epoch summary: mean_loss=0.7406, mean_acc=40.13%
Grad norm: 0.7633

Epoch summary: mean_loss=0.7400, mean_acc=39.41%
Grad norm: 0.6257

Epoch summary: mean_loss=0.7398, mean_acc=40.50%
Grad norm: 0.7333

Epoch summary: mean_loss=0.7377, mean_acc=40.03

In [ ]:
if results_rows:
    df = pd.DataFrame(results_rows)
    tsv_path = os.path.join(folder, "summary_results.tsv")
    df.to_csv(tsv_path, sep='\t', index=False)
    print(f"\n✅ Wrote table summary to {tsv_path}")
    print(df.to_markdown(index=False))


✅ Wrote table summary to results_2025-11-05-16_43/summary_results.tsv
| dataset   | loss       | estimate_T   |   epochs |   runs |   mean_test_acc |   std_test_acc |   mean_train_loss |   std_train_loss |   fro_error |   mae_error |   rre_error |
|:----------|:-----------|:-------------|---------:|-------:|----------------:|---------------:|------------------:|-----------------:|------------:|------------:|------------:|
| cifar     | forward    | anchor       |        5 |      5 |           33.41 |           0.16 |            1.047  |           0.0444 |    nan      |    nan      |    nan      |
| cifar     | gce        | anchor       |        5 |      5 |           33.53 |           1.91 |            0.9311 |           0.0039 |    nan      |    nan      |    nan      |
| cifar     | forwardGCE | anchor       |        5 |      5 |           33.33 |           0    |            0.897  |           0.0342 |    nan      |    nan      |    nan      |
| cifar     | trevision  | anchor      

## Evaluate

In [ ]:
for filename in os.listdir(folder):
    if filename.endswith(".json"):
        filepath = os.path.join(folder, filename)
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
            
            dataset = data.get("dataset", "N/A")
            loss = data.get("loss", "N/A")
            mean_acc = data.get("mean_test_acc", None)
            std_acc = data.get("std_test_acc", None)

            print(f"{filename}:")
            
            print(f"  dataset       = {dataset}")
            print(f"  loss          = {loss}")
            print(f"  mean_test_acc = {mean_acc:.4f}" if mean_acc is not None else "  mean_test_acc = N/A")
            print(f"  std_test_acc  = {std_acc:.4f}" if std_acc is not None else "  std_test_acc = N/A")
            print("-" * 60)

        except Exception as e:
            print(f"Error reading {filename}: {e}")

Error reading fashion03_results.json: 'list' object has no attribute 'get'
Error reading fashion06_results.json: 'list' object has no attribute 'get'
Error reading cifar_results.json: 'list' object has no attribute 'get'


In [ ]:
C = 3
files = os.listdir()
files = sorted(glob.glob('results*'), reverse=True)
folder = files[0]
print(folder)

# pattern for files starting with "name" and ending with ".json"
files = sorted(glob.glob(os.path.join(folder,"fashion03*.json")))
print(files)

# pick the first matching file
first_file = files[0]
print("Loading:", first_file)

# load the JSON contents
with open(first_file, "r", encoding="utf-8") as f:
    data = json.load(f)

T_prime = np.array(data['last_estimated_T'])
T_true = pick_known_T('fashion03')

#checking recreation performance
print(T_prime)
print(T_true)
print(f"Fro error: {np.linalg.norm(T_prime - T_true, 'fro')}")
print(f"rre error: {np.linalg.norm(T_prime - T_true, 'fro') / np.linalg.norm(T_true, 'fro')}")
print(f"mae error: {np.mean(np.abs(T_prime - T_true))}")


corrs = [st.pearsonr(T_true[i], T_prime[i])[0] for i in range(C)]
print("Per-row correlations:", corrs)
print("Mean:", np.mean(corrs))


# pattern for files starting with "name" and ending with ".json"
files = sorted(f for f in glob.glob(os.path.join(folder, "fashion06*.json")) if "None" not in f)


# pick the first matching file
first_file = files[0]
print("\nLoading:", first_file)
# load the JSON contents
with open(first_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print()
T_prime = np.array(data['last_estimated_T'])

T_true = pick_known_T('fashion06')

#checking recreation performance
print(T_prime)
print(T_true)
print(f"Fro error: {np.linalg.norm(T_prime - T_true, 'fro')}")
print(f"rre error: {np.linalg.norm(T_prime - T_true, 'fro') / np.linalg.norm(T_true, 'fro')}")
print(f"mae error: {np.mean(np.abs(T_prime - T_true))}")


corrs = [st.pearsonr(T_true[i], T_prime[i])[0] for i in range(C)]
print("Per-row correlations:", corrs)
print("Mean:", np.mean(corrs))

results_2025-11-05-16_43
['results_2025-11-05-16_43/fashion03_results.json']
Loading: results_2025-11-05-16_43/fashion03_results.json


TypeError: list indices must be integers or slices, not str

In [ ]:
import os
import glob
import json
import numpy as np
from scipy import stats as st

def evaluate_T_matrices(folder, pattern, dataset_name, C=3):
    """
    Loops over all JSON files matching pattern, computes error metrics 
    and correlations vs. known T, and returns best-performing result.
    """
    files = sorted(glob.glob(os.path.join(folder, pattern)))
    if not files:
        print(f"No files found matching: {pattern}")
        return None, None
    
    T_true = pick_known_T(dataset_name)
    results = []

    print(f"Evaluating {len(files)} files for dataset: {dataset_name}\n")

    for file in files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        T_prime = np.array(data['last_estimated_T'])
        
        fro_err = np.linalg.norm(T_prime - T_true, 'fro')
        rre_err = fro_err / np.linalg.norm(T_true, 'fro')
        mae_err = np.mean(np.abs(T_prime - T_true))
        corrs = [st.pearsonr(T_true[i], T_prime[i])[0] for i in range(C)]
        mean_corr = np.mean(corrs)
        
        print(f"File: {os.path.basename(file)}")
        print(f"Fro error: {fro_err:.6f}")
        print(f"RRE error: {rre_err:.6f}")
        print(f"MAE error: {mae_err:.6f}")
        print(f"Per-row correlations: {np.round(corrs, 4)} | Mean: {mean_corr:.4f}")
        print("-" * 60)

        #checking recreation performance
        print("T_prime:")
        print(np.round(T_prime, 2))
        print("T_true:")
        print(np.round(T_true, 2))
        print("T_true - T_prime:")
        print(np.round(T_true - T_prime, 2))
        
        results.append({
            'file': file,
            'T_prime': T_prime,
            'fro': fro_err,
            'rre': rre_err,
            'mae': mae_err,
            'corr_mean': mean_corr
        })
    
    # Choose the best result (lowest Frobenius error)
    best = min(results, key=lambda x: x['fro'])
    print(f"\n✅ Best result: {os.path.basename(best['file'])}")
    print(f"   Fro error: {best['fro']:.6f}, RRE: {best['rre']:.6f}, MAE: {best['mae']:.6f}, Corr: {best['corr_mean']:.4f}")
    
    return best['file'], best['T_prime']


# Example usage

best_file_03, best_T_03 = evaluate_T_matrices(folder, "fashion03*.json", "fashion03", C=3)
best_file_06, best_T_06 = evaluate_T_matrices(folder, "fashion06*True*.json", "fashion06", C=3)


Evaluating 18 files for dataset: fashion03

File: fashion03_forwardGCE_anchor_0.6_2025-11-04-09_11.json
Fro error: 0.726604
RRE error: 0.550837
MAE error: 0.200319
Per-row correlations: [0.6207 0.6522 0.6338] | Mean: 0.6356
------------------------------------------------------------
T_prime:
[[0.7  0.   0.31]
 [0.29 0.69 0.  ]
 [0.02 0.31 0.69]]
T_true:
[[0.7 0.3 0. ]
 [0.  0.7 0.3]
 [0.3 0.  0.7]]
T_true - T_prime:
[[ 0.    0.3  -0.31]
 [-0.29  0.01  0.3 ]
 [ 0.28 -0.31  0.01]]
File: fashion03_forwardGCE_anchor_0.8_2025-11-04-09_11.json
Fro error: 0.730517
RRE error: 0.553804
MAE error: 0.201805
Per-row correlations: [0.6207 0.6472 0.626 ] | Mean: 0.6313
------------------------------------------------------------
T_prime:
[[0.7  0.   0.31]
 [0.29 0.69 0.  ]
 [0.02 0.31 0.69]]
T_true:
[[0.7 0.3 0. ]
 [0.  0.7 0.3]
 [0.3 0.  0.7]]
T_true - T_prime:
[[ 0.    0.3  -0.31]
 [-0.29  0.01  0.3 ]
 [ 0.28 -0.31  0.01]]
File: fashion03_forwardGCE_anchor_0.9_2025-11-04-09_11.json
Fro error: 0.7